In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

In [2]:
# Read the Excel file

UPSGround = pd.read_excel("D:/assignment/4th sem/Solution/UPSGround.xlsx")
USPSGround = pd.read_excel('D:/assignment/4th sem/Solution/USPSGround.xlsx')
FedEx2day = pd.read_csv('D:/assignment/4th sem/Solution/2DayCounter_Fedex.csv',encoding='latin1')
FedExGround = pd.read_csv('D:/assignment/4th sem/Solution/FHDCounter.csv',encoding='latin1')

                        
print(FedExGround.head())
print(FedEx2day.head())
print(USPSGround.head())
print(UPSGround.head()) 

   WeightStartOz  WeightEndOz  Zone 1  Zone 2  Zone 3  Zone 4  Zone 5  Zone 6  \
0              0           16   12.09   12.09   12.59   13.12   13.63   14.09   
1             16           32   12.84   12.84   13.72   14.45   14.68   15.01   
2             32           48   12.85   12.85   13.74   14.47   15.02   15.58   
3             48           64   12.88   12.88   13.77   14.50   16.39   18.82   
4             64           80   12.90   12.90   13.79   14.65   17.27   19.89   

   Zone 7  Zone 8  Zone 9  Zone 10  
0   14.35   14.49   48.92    14.49  
1   15.76   17.11   54.31    17.11  
2   18.04   21.43   59.13    21.43  
3   20.13   22.94   64.32    22.94  
4   21.92   24.03   69.66    24.03  
   WeightStartOz  WeightEndOz  Zone1  Zone 2  Zone 3  Zone 4  Zone 5  Zone 6  \
0              0            8  23.83   23.83   24.84   25.90   30.99   37.22   
1              8           16  23.83   23.83   24.84   25.90   30.99   37.22   
2             16           32  24.80   24.80   26.0

In [3]:
#UPSGround = pd.melt(UPSGround, id_vars=['WeightStartOz', 'WeightEndOz'], var_name='Zone', value_name='Value')
#UPSGround['Carrier'] = 'UPS'
#UPSGround['CarrierService'] = 'UPSGround'



In [4]:
def transform_df(df, additional_columns):
        df_melted = pd.melt(df, id_vars=['WeightStartOz', 'WeightEndOz'], var_name='Zone', value_name='Value')
       # Add additional columns
        for col_name, col_value in additional_columns.items():
            df_melted[col_name] = col_value
    
        return df_melted

In [5]:
FedEx2day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   WeightStartOz  151 non-null    int64  
 1   WeightEndOz    151 non-null    int64  
 2   Zone1          151 non-null    float64
 3   Zone 2         151 non-null    float64
 4   Zone 3         151 non-null    float64
 5   Zone 4         151 non-null    float64
 6   Zone 5         151 non-null    float64
 7   Zone 6         151 non-null    float64
 8   Zone 7         151 non-null    float64
 9   Zone 8         151 non-null    float64
 10  Zone9          151 non-null    float64
 11  Zone10         151 non-null    float64
dtypes: float64(10), int64(2)
memory usage: 14.3 KB


In [6]:
FedEx2day= transform_df(FedEx2day, additional_columns={'Carrier': 'FedEx', 'CarrierService': 'FedEx2day' , 'Shipoption' : '2day'})
FedExGround = transform_df(FedExGround, additional_columns={'Carrier': 'FedEx', 'CarrierService': 'FedExGround','Shipoption' : 'Ground'})
UPSGround = transform_df(UPSGround, additional_columns={'Carrier': 'UPS', 'CarrierService': 'UPSGround','Shipoption' : 'Ground'})
USPSGround = transform_df(USPSGround, additional_columns={'Carrier': 'USPS', 'CarrierService': 'USPSGround','Shipoption' : 'Ground'})


In [7]:
df = pd.concat([FedEx2day, FedExGround, UPSGround, USPSGround])

In [8]:
df.head()

,WeightStartOz,WeightEndOz,Zone,Value,Carrier,CarrierService,Shipoption
0,0,8,Zone1,23.83,FedEx,FedEx2day,2day
1,8,16,Zone1,23.83,FedEx,FedEx2day,2day
2,16,32,Zone1,24.80,FedEx,FedEx2day,2day
3,32,48,Zone1,25.20,FedEx,FedEx2day,2day
4,48,64,Zone1,25.97,FedEx,FedEx2day,2day


In [9]:

from sqlalchemy import create_engine
import pyodbc




# Define your SQL Server connection parameters
server = 'LAPTOP-2MROT4G8\SQLEXPRESS'
database = 'master'
table_name = 'Ratecard'



# Create a connection using pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-2MROT4G8\SQLEXPRESS;'
    'DATABASE=master;'
    'Trusted_Connection=yes;'
)

       




# Create a connection string using Trusted Connection (Windows Authentication)
connection_string = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"

# Create an SQLAlchemy engine using the connection string
engine = create_engine(connection_string)

# Optional: Connect to the database and delete the existing table if it exists
#with engine.connect() as connection:
#    connection.execute(f"DROP TABLE IF EXISTS {table_name}")
#    print(f"Table {table_name} deleted (if it existed).")
#


# Load DataFrame into SQL Server table
df.to_sql(table_name, engine, if_exists='replace', index=False)

print("DataFrame successfully loaded into SQL Server table.")


DataFrame successfully loaded into SQL Server table.
